In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Epilepsy"
cohort = "GSE64123"

# Input paths
in_trait_dir = "../../input/GEO/Epilepsy"
in_cohort_dir = "../../input/GEO/Epilepsy/GSE64123"

# Output paths
out_data_file = "../../output/preprocess/Epilepsy/GSE64123.csv"
out_gene_data_file = "../../output/preprocess/Epilepsy/gene_data/GSE64123.csv"
out_clinical_data_file = "../../output/preprocess/Epilepsy/clinical_data/GSE64123.csv"
json_path = "../../output/preprocess/Epilepsy/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Human embryonic stem cell based neuro-developmental toxicity assay: response to valproic acid and carbamazepine exposure"
!Series_summary	"Here we studied the effects of anticonvulsant drug exposure in a human embryonic stem cell (hESC) based neuro- developmental toxicity test (hESTn). During neural differentiation the cells were exposed, for either 1 or 7 days, to non-cytotoxic concentration ranges of valproic acid (VPA) or carbamazepine (CBZ), anti-epileptic drugs known to cause neurodevelopmental toxicity."
!Series_overall_design	"93 samples (multiple time points, multiple exposures, multiple concentrations, multiple replicates)"
Sample Characteristics Dictionary:
{0: ['time: 0 days', 'time: 1 days', 'time: 4 days', 'time: 7 days', 'time: 9 days', 'time: 11 days'], 1: ['exposure: unexposed', 'exposure: DMSO', 'exposure: carbamazepine', 'exposure: valproic acid'], 2: ['concentration: 0 mM', 'concentration: 0.25%', 'concentration: 0.033 mM', 'con

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
from typing import Optional, Callable, Dict, Any
import json

# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to be about gene expression during neural differentiation
# and the effects of drug exposure, so it likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at sample characteristics dictionary, we don't find direct trait (epilepsy) information
# The dataset is about effects of anticonvulsant drugs on neural development, not patients with epilepsy
trait_row = None  # No epilepsy trait data available

# Age data is not available in the sample characteristics
age_row = None

# Gender data is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> Optional[int]:
    """Convert epilepsy trait value to binary (0/1)"""
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.lower().strip()
        
    if value in ['yes', 'epilepsy', 'epileptic', 'seizure disorder', 'true', '1']:
        return 1
    elif value in ['no', 'control', 'healthy', 'normal', 'false', '0']:
        return 0
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """Convert age value to continuous numeric value"""
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender value to binary (0=female, 1=male)"""
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.lower().strip()
    
    if value in ['female', 'f', 'woman', 'girl']:
        return 0
    elif value in ['male', 'm', 'man', 'boy']:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step as trait_row is None (no clinical data available for our specific trait of interest)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


SOFT file: ../../input/GEO/Epilepsy/GSE64123/GSE64123_family.soft.gz
Matrix file: ../../input/GEO/Epilepsy/GSE64123/GSE64123_series_matrix.txt.gz
Found the matrix table marker in the file.


Gene data shape: (18909, 93)
First 20 gene/probe identifiers:
['100009676_at', '10000_at', '10001_at', '10002_at', '10003_at', '100048912_at', '100049716_at', '10004_at', '10005_at', '10006_at', '10007_at', '10008_at', '100093630_at', '10009_at', '1000_at', '100101467_at', '100101938_at', '10010_at', '100113407_at', '10011_at']


### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers in the provided list
# The format "100009676_at" suggests these are Affymetrix microarray probe set IDs
# These are not standard human gene symbols and need to be mapped to gene symbols
# Affymetrix IDs typically end with "_at" and need conversion to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Based on the preview, 'ID' appears to be the probe ID and 'Description' contains gene names
# Display more samples from the Description column to better understand the format
print("\nSample of Description column (first 5 rows):")
if 'Description' in gene_annotation.columns:
    for i in range(min(5, len(gene_annotation))):
        print(f"Row {i}: {gene_annotation['Description'].iloc[i]}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'SPOT_ID', 'Description']
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'SPOT_ID': ['1', '10', '100', '1000', '10000'], 'Description': ['alpha-1-B glycoprotein', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'adenosine deaminase', 'cadherin 2, type 1, N-cadherin (neuronal)', 'v-akt murine thymoma viral oncogene homolog 3 (protein kinase B, gamma)']}

Sample of Description column (first 5 rows):
Row 0: alpha-1-B glycoprotein
Row 1: N-acetyltransferase 2 (arylamine N-acetyltransferase)
Row 2: adenosine deaminase
Row 3: cadherin 2, type 1, N-cadherin (neuronal)
Row 4: v-akt murine thymoma viral oncogene homolog 3 (protein kinase B, gamma)


### Step 6: Gene Identifier Mapping

In [7]:
# Analyze the identifiers in gene annotation and gene expression data
print("Example probe IDs in gene expression data:", gene_data.index[:5].tolist())
print("Example IDs in annotation data:", gene_annotation['ID'][:5].tolist())

# 1. Determine which columns to use for mapping
# The gene annotation 'ID' column contains probe IDs like "1_at"
# The gene expression data index contains probes like "100009676_at"
# The 'Description' column contains gene symbols/descriptions

# 2. Create the gene mapping dataframe
# Extract the relevant columns for mapping
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Description')
print("Gene mapping preview:")
print(preview_df(gene_mapping, n=5))
print(f"Shape of gene mapping dataframe: {gene_mapping.shape}")

# Count how many probes can be mapped to the gene expression data
common_probes = set(gene_data.index).intersection(set(gene_mapping['ID']))
print(f"Number of probes in gene expression data that can be mapped: {len(common_probes)}")

# 3. Apply gene mapping to convert probe-level data to gene-level expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print("Gene expression data after mapping:")
print(f"Shape of gene expression data: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10].tolist())

# Apply standardization to gene symbols
gene_data = normalize_gene_symbols_in_index(gene_data)
print("\nGene expression data after normalizing gene symbols:")
print(f"Shape of gene expression data: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10].tolist())

# Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Example probe IDs in gene expression data: ['100009676_at', '10000_at', '10001_at', '10002_at', '10003_at']
Example IDs in annotation data: ['1_at', '10_at', '100_at', '1000_at', '10000_at']
Gene mapping preview:
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'Gene': ['alpha-1-B glycoprotein', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'adenosine deaminase', 'cadherin 2, type 1, N-cadherin (neuronal)', 'v-akt murine thymoma viral oncogene homolog 3 (protein kinase B, gamma)']}
Shape of gene mapping dataframe: (18876, 2)
Number of probes in gene expression data that can be mapped: 18876
Gene expression data after mapping:
Shape of gene expression data: (2024, 93)
First few gene symbols:
['A-', 'A-2', 'A-52', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1', 'A10']



Gene expression data after normalizing gene symbols:
Shape of gene expression data: (1168, 93)
First few normalized gene symbols:
['A1BG', 'A4GALT', 'AAA1', 'ABCC11', 'ABCD1', 'ABCE1', 'ABI3', 'ABO', 'ACSM3', 'ADAT2']


Gene expression data saved to ../../output/preprocess/Epilepsy/gene_data/GSE64123.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data - respecting the analysis from Step 2
# From Step 2, we determined:
# trait_row = None  # No Epilepsy data available
# age_row = None
# gender_row = None
is_trait_available = trait_row is not None

# Skip clinical feature extraction when trait_row is None
if is_trait_available:
    try:
        # Load the clinical data from file
        soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        background_info, clinical_data = get_background_and_clinical_data(matrix_file)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            gender_row=gender_row,
            convert_gender=convert_gender,
            age_row=age_row,
            convert_age=convert_age
        )
        
        print(f"Extracted clinical data shape: {clinical_features.shape}")
        print("Preview of clinical data (first 5 samples):")
        print(clinical_features.iloc[:, :5])
        
        # Save the properly extracted clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical data: {e}")
        is_trait_available = False
else:
    print("No trait data (Epilepsy) available in this dataset based on previous analysis.")

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from GBM cell cultures, but no epilepsy phenotype information."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # Create an empty DataFrame for metadata purposes
    empty_df = pd.DataFrame()
    
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=empty_df,  # Empty dataframe for metadata
        note="Missing essential data components for linking: dataset contains gene expression data from GBM cell cultures, but no epilepsy phenotype information."
    )

Gene data shape before normalization: (1168, 93)
Gene data shape after normalization: (1168, 93)


Normalized gene data saved to ../../output/preprocess/Epilepsy/gene_data/GSE64123.csv
No trait data (Epilepsy) available in this dataset based on previous analysis.
Cannot proceed with data linking due to missing trait or gene data.
Abnormality detected in the cohort: GSE64123. Preprocessing failed.
